In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import scrublet as scr
import scipy.io
import matplotlib.pyplot as plt
import os
import scvelo as scv
import harmonypy as hm
from matplotlib.pyplot import rc_context
import re
import seaborn as sns
from adpbulk import ADPBulk
import diffxpy.api as de


In [ ]:
sc.settings.set_figure_params(dpi=150,fontsize=8)

# Load integrated and harmonized data

In [ ]:
datapath = r"/nfs_master/prakrithi/abhay/testis_scripts/"

In [ ]:
adata = sc.read_h5ad(datapath+'integrated_data.h5ad')
adata.uns['log1p']["base"] = None

In [ ]:
adata

In [ ]:
!mkdir results_Nov25

# Add metadata for TE elements


In [ ]:
#load list of human TEs
all_te = pd.read_csv('/nfs_master/prakrithi/abhay/testis_scripts/extra_files/all_TE.csv')
all_te = all_te['All_TEs'].tolist()
gene_list = list(adata.var.index)
te_list = list(set(all_te).intersection(gene_list))

In [ ]:
#Alu = list(filter(lambda x:'Alu' in x, all_te))
#Alu

In [ ]:
Alu  = list(filter(lambda x:'Alu' in x, te_list))
AluY = list(filter(lambda x:'AluY' in x, te_list))
L1   = list(filter(lambda x:'L1' in x, te_list))
LINE = list(filter(lambda x:'LINE' in x, te_list))
LTR  = list(filter(lambda x:'LTR' in x, te_list))
SVA  = list(filter(lambda x:'SVA' in x, te_list))

In [ ]:
len(te_list)


# Save raw data

In [ ]:
adata.raw = adata

# Dimensionality Reduction

#### Calculate highly variable genes

In [ ]:
sc.pp.highly_variable_genes(adata,n_top_genes=3000)

In [ ]:
adata = adata[:, adata.var.highly_variable]

In [ ]:
adata.var

In [ ]:
sc.pp.regress_out(adata, ['total_counts', 'percent_mito', 'percent_ribo'])     



In [ ]:
sc.pp.scale(adata, max_value=10)

In [ ]:
sc.tl.pca(adata, svd_solver='arpack', n_comps=40)

In [ ]:
sc.pl.pca_variance_ratio(adata, log=False)

#### Harmony Batch correction

In [ ]:
pca = adata.obsm['X_pca']
batch = adata.obs['samples']
meta_data = adata.obs

In [ ]:
ho = hm.run_harmony(pca, meta_data, ['samples'], epsilon_harmony = -float('Inf'), max_iter_kmeans = 25)


In [ ]:
res = pd.DataFrame(ho.Z_corr)
res = res.T
adata_hvg.obsm['X_pca'] = res.values


# Clustering

In [ ]:
sc.pp.normalize_total(adata_hvg, target_sum=1e4)
sc.pp.log1p(adata)
adata.raw=adata

In [ ]:
sc.pp.neighbors(adata_hvg,n_neighbors=30)

In [ ]:
sc.tl.umap(adata_hvg,spread=0.8,min_dist=0.3)

In [ ]:
sc.tl.leiden(adata_hvg,resolution=0.45, n_iterations=-1)

In [ ]:
sc.pl.umap(adata_hvg,color=["leiden","samples"],cmap="plasma_r")
plt.savefig('integrated-umap.png')

# Differential Gene Expression Analysis

In [ ]:
sc.tl.rank_genes_groups(adata_hvg,pts=True,groupby='leiden',n_genes=100,method='wilcoxon',corr_method='bonferroni')

In [ ]:
sc.tl.dendrogram(adata_hvg,groupby="leiden")

In [ ]:
with rc_context({'figure.figsize': (8, 8)}):
    sc.pl.rank_genes_groups_dotplot(adata_hvg,n_genes=7,
                                    groupby="leiden",color_map="plasma_r")

In [ ]:
markers = sc.get.rank_genes_groups_df(adata_hvg, None)
markers = markers[(markers.pvals_adj < 0.05) & (markers.logfoldchanges > 0.5)]
markers.to_csv('DEG_by_cluster.csv')


In [ ]:
deg_df = pd.read_csv('DEG_by_cluster.csv')
deg_df = deg_df.iloc[: , 1:]
deg_df = deg_df[(abs(deg_df.logfoldchanges) > 0.25) & (deg_df.pvals_adj < 0.05)]
deg_df

# Cluster Annotation  

In [ ]:
sc.pl.umap(adata, color=["leiden", "samples"], frameon = False)
plt.savefig('unannotated-umap.png')

In [ ]:
#markers of interest
macrophage         = ['CD14', 'CD163', 'S100A4']     
endothelial        = ['PALMD', 'VWF', 'CDH5']        
other              = ['RBP1', 'INSL3', 'MYL9']       
lydig              = ['DLK1','IGF1','CFD']           
myoid              = ['MYH11','ACTA2','TPM4']        
sertoli            = ['SOX9', 'WT1', 'HMGN5']        
#immature_spermatid = ['ZPBP', 'ZPBP2', 'SPAG6']
#spermatid          = ['SPATA18', 'HOOK1', 'SPATA12'] 

#spermatogonia
spermatogonia      = ['UTF1', 'ID4', 'SOHLH1']  
undiff_SPG         = ['MAGEA4','ID4','TCF3']
diff_SPG           = ['SOX4','DMRT1']

#speermatocytes
leptotene          = ['TEX19','DPH7','DMC1']
zygotene           = ['MLH3','SELENOT', 'TDRG1','LY6K']
pachytene          = ['PIWIL1','CCDC112']
diplotene          = ['AURKA', 'CCNA1']
meotic_div         = ['SIRPG', 'SLC26A3']

round_spt          = ['SPAG6']
elong_spt          = ['ZPBP', 'ZPBP2']
#elong_spt          = ['ZPBP', 'ZPBP2', 'DNAH6', 'DNAH7']
early_spt          = ['TEX29']
late_spt           = ['PRM3','SPATA12']


In [ ]:
#marker lists
spermatogonia = undiff_SPG + diff_SPG
spermatids = round_spt + elong_spt + early_spt + late_spt
pre_pachytene = leptotene + zygotene 
post_pachytene = pachytene + diplotene 

somatic_m = macrophage + lydig + sertoli + endothelial
 
best = spermatids + post_pachytene + spermatogonia + pre_pachytene + sertoli + lydig + endothelial + macrophage


In [ ]:
adata_hvg.obs

In [ ]:
label_dict = {'0': 'Elongating_spermatids',
              '1': 'Lydig_cells',
              '2': 'Round_spermatids',
              '3': "Post_pachytene_spermatocytes",
              '4': "Mature_spermatids",
              '5': "Unknown_1",
              '6': "Spermatogonia",
              '7': "Pre_pachytene_spermatocytes",
              '8': "Sertoli_cells",
              '9': "Endothelial_cells",
            '10' : "Macrophages",
            '11' : "Unknown_2",
            '12' : "Unknown_3"
             } 
adata_hvg.obs['clusters'] = adata_hvg.obs['leiden'].map(label_dict).astype('category')

In [ ]:
#test marker dotplot for annotation
sc.pl.dotplot(adata_hvg, Alu, groupby='clusters', dendrogram=True)
plt.savefig('test-dotplot.png')

In [ ]:
adata.var.index

In [ ]:
#labelled dotplot
sc.pl.dotplot(adata_hvg, best, groupby='clusters', dendrogram=True)
plt.savefig('labelled-dotplot.png')

In [ ]:
#labelled UMAP
sc.pl.umap(adata_hvg, color=["clusters"], frameon = False, legend_loc = 'on data', legend_fontsize = 'xx-small')
plt.savefig('labelled-umap.png')

# Subset germcells

In [ ]:
germcells = adata_hvg[adata_hvg.obs['leiden'].isin(['0', '2', '3', '4', '6', '7'])].copy()
germcells



In [ ]:
germcells.write_h5ad('germcells.h5ad')

In [ ]:
germcells.obs

# Subset spermatocytes

In [ ]:
spermatocytes = adata_hvg[adata_hvg.obs['leiden'].isin(['3', '7'])]

In [ ]:
spermatocytes.obs

# Analysis 

In [ ]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [ ]:
def map_condition(x):
    if "Donor" in x:
        return "Normal"
    elif "Normal" in x:
        return "Normal"
    elif "Crypto" in x:
        return "Cryptozoospermia"
    elif "iNOA" in x:
        return "Non-obstructive_azoospermia"
    else:
        return "obstructive_azoospermia"

In [ ]:
adata_hvg.obs['condition'] = adata_hvg.obs.samples.map(map_condition)
adata_hvg.obs

In [ ]:
num_tot_cells = adata_hvg.obs.groupby(['samples']).count()
num_tot_cells = dict(zip(num_tot_cells.index, num_tot_cells.percent_TE))

num_tot_cells

In [ ]:
cell_type_counts = adata_hvg.obs.groupby(['samples','condition', 'clusters']).count()
cell_type_counts = cell_type_counts[cell_type_counts.sum(axis =1) > 0].reset_index()
cell_type_counts = cell_type_counts[cell_type_counts.columns[0:4]]
cell_type_counts = cell_type_counts.rename(columns = {'percent_TE':'cell_count'})

# cell_type_counts.to_csv('germcell_count.csv')

cell_type_counts

In [ ]:
cell_type_counts['total_cells'] = cell_type_counts.samples.map(num_tot_cells).astype(int)

cell_type_counts['frequency'] = cell_type_counts.cell_count/ cell_type_counts.total_cells

cell_type_counts

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (15,7))

ax = sns.boxplot(data = cell_type_counts, x = 'clusters', y = 'frequency', hue = 'condition')

plt.xticks(rotation = 35, rotation_mode = 'anchor', ha = 'right')

plt.show()
plt.savefig('integrated_freq_plot')

# DGE of germcells 

In [ ]:
germcells.obs['condition'] = germcells.obs.samples.map(map_condition)
germcells.obs

In [ ]:
sc.tl.rank_genes_groups(germcells,pts=True,groupby='condition',n_genes=100,method='wilcoxon',corr_method='bonferroni')

In [ ]:
sc.tl.dendrogram(germcells,groupby="condition")

In [ ]:
TE_of_interest = intersection(te_list, adata_hvg.var.index)
TE_of_interest

In [ ]:
#Alu expression by condition in gremcells
sc.pl.dotplot(germcells, TE_of_interest, groupby='condition', dendrogram=True)
plt.savefig('germcell-alu-dotplot.png')

### DGE of spermatocytes by condition

In [ ]:
sc.tl.rank_genes_groups(spermatocytes,pts=True,groupby='condition',n_genes=100,method='wilcoxon',corr_method='bonferroni')

In [ ]:
sc.tl.dendrogram(spermatocytes,groupby="condition")

In [ ]:
spermatocytes.obs['condition'] = spermatocytes.obs.samples.map(map_condition)
spermatocytes.obs

In [ ]:
#Alu expression by condition in spermatocytes
sc.pl.dotplot(germcells, TE_of_interest, groupby='condition', dendrogram=True)
plt.savefig('spermatocytes-alu-dotplot.png')

# Pseudobulking

In [ ]:
#initialize
adpb_sum  = ADPBulk(adata, ["leiden", "samples"], "sum")
adpb_mean = ADPBulk(adata, ["leiden", "samples"], "mean")

# perform the pseudobulking
pseudobulk_matrix_sum  = adpb_sum.fit_transform()
pseudobulk_matrix_mean = adpb_mean.fit_transform()

# retrieve the sample meta data (useful for easy incorporation with edgeR)
sample_meta_sum  = adpb_sum.get_meta()
sample_meta_mean = adpb_mean.get_meta()



In [ ]:
pseudobulk_matrix_sum

In [ ]:
pseudobulk_matrix_mean

In [ ]:
pseudobulk_matrix = pseudobulk_matrix_mean.to_csv("integrated_pseudobulk_matrix.csv")

In [ ]:
sample_meta_summ

# WCGNA